In [ ]:
from tqdm.auto import tqdm

from utils.io import enum_files
from rule.distance_SAQ import Dis
from rule.distance_compare_TFQ import DistanceCompareTFQGenerator
from rule.count_SAQ import CountSAQGenerator
from rule.volume_SAQ import VolumeSAQGenerator

In [ ]:
for scene_json in tqdm(
    enum_files('../NUMINA-dataset/ScanNet-scene_stats'),
    desc='Processing'
):
    DistanceCompareTFQGenerator(
        scene_stat_json_file=scene_json,
        export_json_file='../output/NUM-distance-SAQ.json',
        excluded_labels=['ceiling', 'floor', 'wall', 'object'],
    ).generate(n_questions=6, enforce_unique_pairs=True)

In [ ]:
for scene_json in tqdm(
    enum_files('../NUMINA-dataset/ScanNet-scene_stats'),
    desc='Processing'
):
    DistanceSAQGenerator(
        scene_stat_json_file=scene_json,
        export_json_file='../output/NUM-distance-SAQ.json',
        excluded_labels=['ceiling', 'floor', 'wall', 'object'],
    ).generate(n_questions=10, enforce_unique_pairs=True)

In [ ]:
for scene_json in tqdm(
    enum_files('../NUMINA-dataset/ScanNet-scene_stats'),
    desc='Processing'
):
    CountSAQGenerator(
        scene_stat_json_file=scene_json,
        export_json_file='../output/NUM-count-SAQ.json',
        excluded_labels=['ceiling', 'floor', 'wall', 'object'],
    ).generate(n_questions=10, enforce_unique_labels=True)

In [ ]:
for scene_json in tqdm(
    enum_files('../NUMINA-dataset/ScanNet-scene_stats'),
    desc='Processing'
):
    VolumeSAQGenerator(
        scene_stat_json_file=scene_json,
        export_json_file='../output/NUM-volume-SAQ.json',
        excluded_labels=['ceiling', 'floor', 'wall', 'object'],
    ).generate(n_questions=10, enforce_unique_labels=True)